##Importing the required libraries

In [ ]:
import nltk
nltk.download('stopwords')
from spacy.lang.en import English
from spacy.lang.en.stop_words import STOP_WORDS
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
#import pyLDAvis
#import pyLDAvis.gensim  # don't skip this
!pip install pyLDAvis

import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
#vis = pyLDAvis.gensim_models.prepare(ldamodel, doc_term_matrix, dictionary)

import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
import nltk
nltk.download('all')

##Uploading the json files

In [2]:
import json 
import numpy as np
import csv 
from google.colab import files
uploaded = files.upload()

Saving thehindu_jan_1_file_2.json to thehindu_jan_1_file_2.json
Saving thehindu_jan_1_file_3.json to thehindu_jan_1_file_3.json
Saving thehindu_jan_1_file_4.json to thehindu_jan_1_file_4.json
Saving thehindu_jan_1_file_6.json to thehindu_jan_1_file_6.json
Saving thehindu_jan_1_file_7.json to thehindu_jan_1_file_7.json
Saving thehindu_jan_1_file_9.json to thehindu_jan_1_file_9.json
Saving thehindu_jan_1_file_10.json to thehindu_jan_1_file_10.json
Saving thehindu_jan_1_file_11.json to thehindu_jan_1_file_11.json
Saving thehindu_jan_1_file_12.json to thehindu_jan_1_file_12.json
Saving thehindu_jan_1_file_13.json to thehindu_jan_1_file_13.json


In [3]:
#Creating a list of names of all the json files 

name_list = []

def getList(dict):
    return list(dict.keys())

name_list = getList(uploaded)
name_list

['thehindu_jan_1_file_2.json',
 'thehindu_jan_1_file_3.json',
 'thehindu_jan_1_file_4.json',
 'thehindu_jan_1_file_6.json',
 'thehindu_jan_1_file_7.json',
 'thehindu_jan_1_file_9.json',
 'thehindu_jan_1_file_10.json',
 'thehindu_jan_1_file_11.json',
 'thehindu_jan_1_file_12.json',
 'thehindu_jan_1_file_13.json']

In [4]:
#Extracting data from the json files

data_list=[]
for i in name_list:
  data_list.append(uploaded[i].decode("utf-8"))
data_list

['{"Text": "\\nNBA | Wall makes Rockets debut in win over Kings\\n\\nRaptors ends worst start to season in 15 years with defeat of New York Knicks\\n\\nBack with a bang: Houston Rockets guard John Wall made his return to action count against Sacramento Kings.\\n\\u00a0 | Photo Credit: Richard Carson James Harden scored a game high 33 points and John Wall finished with a double-double in his first game since 2018 as Houston Rockets beat Sacramento Kings 122-119 on Thursday.Wall tallied 22 points and nine assists for the Rockets, who were also playing their first game with DeMarcus Cousins and Eric Gordon. Wall was acquired from Washington Wizards for Russell Westbrook.Harden sank two free-throws late in the game as the Rockets held on for the win. Kings had a chance to tie it but missed two threes in the final seconds in front of a crowd of 3,200 in Texas.Harden also had eight assists and six rebounds in the first game of a two-game set with the Kings. Elsewhere, Joel Embiid delivered 2

In [6]:
#Creating a dataframe with the raw and processed text to analyze differences

import pandas as pd
df = pd.DataFrame(zip(data_list, data_list),
               columns =['Raw_Text', 'Processed_Text'])
df.head(len(name_list))

,Raw_Text,Processed_Text
0,"{""Text"": ""\nNBA | Wall makes Rockets debut in ...","{""Text"": ""\nNBA | Wall makes Rockets debut in ..."
1,"{""Text"": ""\nDecisive, Glorious Land, King T\u2...","{""Text"": ""\nDecisive, Glorious Land, King T\u2..."
2,"{""Text"": ""\nBorn Queen takes South India Oaks ...","{""Text"": ""\nBorn Queen takes South India Oaks ..."
3,"{""Text"": ""\nIndiGo servers hacked in December;...","{""Text"": ""\nIndiGo servers hacked in December;..."
4,"{""Text"": ""\nSony to launch PlayStation 5 in In...","{""Text"": ""\nSony to launch PlayStation 5 in In..."
5,"{""Text"": ""\nApple removes 39,000 game apps fro...","{""Text"": ""\nApple removes 39,000 game apps fro..."
6,"{""Text"": ""\nMicrosoft says hackers viewed sour...","{""Text"": ""\nMicrosoft says hackers viewed sour..."
7,"{""Text"": ""\nTop news of the day: India may app...","{""Text"": ""\nTop news of the day: India may app..."
8,"{""Text"": ""\nAlgerian president signs new const...","{""Text"": ""\nAlgerian president signs new const..."
9,"{""Text"": ""\nDelay sought in ex-officers\u2019 ...","{""Text"": ""\nDelay sought in ex-officers\u2019 ..."


##Data Cleansing

In [9]:
for m in range(len(name_list)):
  #Removing new line commands like \n, \t, \r
  df['Processed_Text'].values[m] = (re.sub(r"\\n", " ", df['Processed_Text'].values[m]))
  df['Processed_Text'].values[m] = (re.sub(r"\\u", " ", df['Processed_Text'].values[m]))
  df['Processed_Text'].values[m] = (re.sub(r"\\t", " ", df['Processed_Text'].values[m]))
  df['Processed_Text'].values[m] = (re.sub(r"\\r", " ", df['Processed_Text'].values[m]))

  #Removing any remaining words starting with '/'
  df['Processed_Text'].values[m] = " ".join(filter(lambda x:x[0]!="\\", df['Processed_Text'].values[m].split()))

  #Removing all non-letters and non-digits
  df['Processed_Text'].values[m] = (re.sub(r"\W", " ", df['Processed_Text'].values[m]))
  
  #Removing all numbers
  df["Processed_Text"].replace('\d+', '', regex=True, inplace = True)

  #Removing all extra white spaces
  df['Processed_Text'].values[m] = " ".join(df['Processed_Text'].values[m].split())

  #Removing the first word of each text file, i.e., name of key
  df['Processed_Text'].values[m] = df['Processed_Text'].values[m].split(' ', 1)[1]
  
  print(df['Processed_Text'].values[m])

NBA Wall makes Rockets debut in win over Kings Raptors ends worst start to season in years with defeat of New York Knicks Back with a bang Houston Rockets guard John Wall made his return to action count against Sacramento Kings a Photo Credit Richard Carson James Harden scored a game high points and John Wall finished with a double double in his first game since as Houston Rockets beat Sacramento Kings on Thursday Wall tallied points and nine assists for the Rockets who were also playing their first game with DeMarcus Cousins and Eric Gordon Wall was acquired from Washington Wizards for Russell Westbrook Harden sank two free throws late in the game as the Rockets held on for the win Kings had a chance to tie it but missed two threes in the final seconds in front of a crowd of in Texas Harden also had eight assists and six rebounds in the first game of a two game set with the Kings Elsewhere Joel Embiid delivered points and nine rebounds as Philadelphia ers sank three pointers to hand O

##Stop Words Removal & Tokenizing

In [10]:
from spacy.lang.en import English
from spacy.lang.en.stop_words import STOP_WORDS
STOP_WORDS |= {"the", "jul", "pm", "https", "www", "printable", "version", "thehindu", "com", "news", "ece", 
               "s", "t", "d", "m", "article", "b", "rb","cwe", "click", "subscribe", "photo", "credit"}
# Load English tokenizer, tagger, parser, NER and word vectors
nlp = English()
for m in range(len(name_list)):
  text =df['Processed_Text'].values[m]
    #"nlp" Object is used to create documents with linguistic annotations.
  my_doc = nlp(text)

  # Create list of word tokens
  token_list = []
  for token in my_doc:
    token_list.append(token.text)

  # Create list of word tokens after removing stopwords
  filtered_sentence =[] 

  for word in token_list:
   lexeme = nlp.vocab[word]
   if lexeme.is_stop == False:
        filtered_sentence.append(word) 
  r = filtered_sentence
  df['Processed_Text'].values[m] = r

##Stemming

In [ ]:
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
lancaster=LancasterStemmer()
ps = PorterStemmer()


for m in range(len(name_list)):
  temp1 = []
  temp2 = df['Processed_Text'].values[m]
  for i in temp2:
    temp1.append(lancaster.stem(i))
  df['Processed_Text'].values[m] = temp1 
  

##Lemmatization

In [11]:
from nltk.stem import WordNetLemmatizer  
lemmatizer = WordNetLemmatizer()

for m in range(len(name_list)):
  temp1 = []
  temp2 = df['Processed_Text'].values[m]
  for i in temp2:
    temp1.append(lemmatizer.lemmatize(i))
  df['Processed_Text'].values[m] = temp1 
df.head(len(name_list))

,Raw_Text,Processed_Text
0,"{""Text"": ""\nNBA | Wall makes Rockets debut in ...","[NBA, Wall, make, Rockets, debut, win, Kings, ..."
1,"{""Text"": ""\nDecisive, Glorious Land, King T\u2...","[Decisive, Glorious, Land, King, Chala, Cape, ..."
2,"{""Text"": ""\nBorn Queen takes South India Oaks ...","[Born, Queen, take, South, India, Oaks, style,..."
3,"{""Text"": ""\nIndiGo servers hacked in December;...","[IndiGo, server, hacked, December, internal, d..."
4,"{""Text"": ""\nSony to launch PlayStation 5 in In...","[Sony, launch, PlayStation, India, February, P..."
5,"{""Text"": ""\nApple removes 39,000 game apps fro...","[Apple, remove, game, apps, China, store, meet..."
6,"{""Text"": ""\nMicrosoft says hackers viewed sour...","[Microsoft, say, hacker, viewed, source, code,..."
7,"{""Text"": ""\nTop news of the day: India may app...","[day, India, approve, Covishield, emergency, u..."
8,"{""Text"": ""\nAlgerian president signs new const...","[Algerian, president, sign, new, constitution,..."
9,"{""Text"": ""\nDelay sought in ex-officers\u2019 ...","[Delay, sought, ex, officer, trial, George, Fl..."


##Bigram Formation

In [ ]:
from gensim.models import Phrases
from gensim.models.phrases import Phraser

#Appending all text lists into a single list-of-lists
sentence_stream = []
for i in range(len(name_list)):
  sentence_stream.append(df['Processed_Text'].values[i])

#Creating a bigram model
bigram = Phrases(sentence_stream, min_count=2, threshold=5, delimiter=b' ')

#Training the model
bigram_phraser = Phraser(bigram)

#Applying the model in each corpus
tokens= []
for x in sentence_stream:
  tokens.append(bigram_phraser[x])


In [16]:
#Checking to see the bigrams formed

for i in range(len(name_list)):
  print(tokens[i])

['NBA', 'Wall', 'make', 'Rockets', 'debut', 'win', 'Kings', 'Raptors', 'end', 'worst', 'start', 'season', 'year', 'defeat', 'New', 'York', 'Knicks', 'bang', 'Houston Rockets', 'guard', 'John', 'Wall', 'return', 'action', 'count', 'Sacramento Kings', 'Richard', 'Carson', 'James', 'Harden', 'scored', 'game', 'high', 'point', 'John', 'Wall', 'finished', 'double', 'double', 'game', 'Houston Rockets', 'beat', 'Sacramento Kings', 'Thursday', 'Wall', 'tallied', 'point', 'assist', 'Rockets', 'playing', 'game', 'DeMarcus', 'Cousins', 'Eric', 'Gordon', 'Wall', 'acquired', 'Washington', 'Wizards', 'Russell', 'Westbrook', 'Harden', 'sank', 'free', 'throw', 'late', 'game', 'Rockets', 'held', 'win', 'Kings', 'chance', 'tie', 'missed', 'three', 'final', 'second', 'crowd', 'Texas', 'Harden', 'assist', 'rebound', 'game', 'game', 'set', 'Kings', 'Joel', 'Embiid', 'delivered', 'point', 'rebound', 'Philadelphia', 'er', 'sank', 'pointer', 'hand', 'Orlando', 'Magic', 'loss', 'new', 'season', 'win', 'Embiid'

In [13]:
#Updating the Processed_Text values in the Data Frame

for i in range(len(name_list)):
  df['Processed_Text'].values[i] = tokens[i]

##Final Output: Clean text ready for Topic Modelling

In [15]:
df.head(len(name_list))

,Raw_Text,Processed_Text
0,"{""Text"": ""\nNBA | Wall makes Rockets debut in ...","[NBA, Wall, make, Rockets, debut, win, Kings, ..."
1,"{""Text"": ""\nDecisive, Glorious Land, King T\u2...","[Decisive, Glorious Land, King Chala, Cape Cod..."
2,"{""Text"": ""\nBorn Queen takes South India Oaks ...","[Born Queen, take, South India, Oaks, style, R..."
3,"{""Text"": ""\nIndiGo servers hacked in December;...","[IndiGo, server hacked, December, internal doc..."
4,"{""Text"": ""\nSony to launch PlayStation 5 in In...","[Sony, launch, PlayStation, India, February, P..."
5,"{""Text"": ""\nApple removes 39,000 game apps fro...","[Apple, remove, game apps, China, store, meet,..."
6,"{""Text"": ""\nMicrosoft says hackers viewed sour...","[Microsoft, say, hacker, viewed, source, code,..."
7,"{""Text"": ""\nTop news of the day: India may app...","[day, India, approve, Covishield, emergency, u..."
8,"{""Text"": ""\nAlgerian president signs new const...","[Algerian, president, sign, new constitution, ..."
9,"{""Text"": ""\nDelay sought in ex-officers\u2019 ...","[Delay, sought, ex, officer, trial, George, Fl..."
